# Pima Diabets - EDA

* Bu veri seti aslen Ulusal Diyabet ve Sindirim ve Böbrek Hastalıkları Enstitüsünden alınmıştır. Veri setinin amacı, veri setine dahil edilen belirli tanısal ölçümlere dayalı olarak, bir hastanın diyabetli olup olmadığını tanısal olarak tahmin etmektir. Bu örneklerin daha büyük bir veritabanından seçilmesine birkaç kısıtlama getirildi. Özellikle, buradaki tüm hastalar, Pima Kızılderili mirasından en az 21 yaşında olan kadınlardır.

* Veri kümeleri, birkaç tıbbi öngörücü değişken ve bir hedef değişken, Sonuçtan oluşur. Öngörücü değişkenler, hastanın sahip olduğu gebelik sayısını, BMI'sini, insülin düzeyini, yaşını vb. İçerir.

* Veri kümesi asında bağlı olan öznitelikler kapsamında hedef değişkenin ne kadar değişip,değişmeyeceği veyahut hedef değişkenin bu öznitelikler tarafından ne kadar etkilenip,etkilenmeyeceğini tanımlamamıza yardımcı olacaktır.
<BR>
* a.**Pregnancies**: Hamile olma sayısı
* b.**Glucose**: Oral glikoz tolerans testinde 2 saatlik plazma glikoz konsantrasyonu
* c.**BloodPressure**: Diastolic kan basıncı (mm Hg)
* d.**SkinThickness**: Triceps cilt kıvrım kalınlığı (mm)
* e.**Insulin**: 2 saatlik insulin değeri (mu U/ml)
* f.**BMI**: Vücut Kitle Endeksi (kilo in kg/(yükseklik in m)^2)
* g.**DiabetesPedigreeFunction**: Hastalık Teşhisi için gerekli soyağacı işlevi
* h.**Age**: Yaş (Yıl)
* j.**Outcome**: Hedef Değişken (0 veya 1) - Diabet hatalığı varsa 1 , yoksa 0

Kütüphanelerimizi yükleyelim

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from palettable.colorbrewer.qualitative import Pastel1_7
from scipy import stats
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
pima = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
pima.head()

* pima olarak adlandırılan df'imizin kopyasını tutalım.Herhangi bir eksiklik veya yanlışlık olursa orginal df'i değiştirmek yerine kopyası üzerinden ilerlemek her zaman avantaj sağlar

In [ ]:
df = pima.copy()

# 1. Veri Seti Hikayesi ve Yapısının İncelenmesi

In [ ]:
df.head()

* dataframe kaç gözlem ve kaç öznitelikten oluşuyor,bunları inceleyelim

In [ ]:
df.shape

* df içerisindeki özniteliklerin ve içerisinde barındırdıkları verilerin tiplerini kontrol edelim

In [ ]:
df.info()

* df'in sahip olduğu öznitelikler için istatistiksel bir gözlem yapalım

In [ ]:
df.describe().T

* df'in içerisinde boş değerlerin olup-olmadığını ve bunların hangi özniteliklerle ilgili olup olmadığını kontrol edelim

In [ ]:
msno.matrix(df)

* df'in içerisinde hiç boş değer yok gibi görünüyor.Kontrol amaçlı df içerisine null().sum() çekelim

In [ ]:
df.isnull().sum()

* Özellikler arasındaki korelasyonu inceleyelim

In [ ]:
f, ax = plt.subplots(figsize=[20, 15])
sns.heatmap(df.corr(), annot=True, ax=ax, cmap="Blues")

* Veri kümesi içerisindeki korelasyona bakacak olursak;
1. Insulin ve SkinThickness arasında 44% pozitif korelasyon,
2. BMI ve SkinThickness arasında 39% pozitif korelasyon,
3. Outcome ve Glucose öznitelikleri arasında 47% pozitif korelasyon,
4. Age ve Pregnancies öznitelikleri arasında 54% pozitif korelasyon,
5. Insulin ve Glucose öznitelikleri arasında 33% pozitif korelasyon bulunmaktadır.

# 2. Veri Seti İçerisindeki Öznitelikleri İnceleyelim

In [ ]:
df.head(10)

In [ ]:
visualized_graph = df.hist(figsize=(20, 20))
visualized_graph

## 2.1. Pregnancies

In [ ]:
df[["Pregnancies"]].head()

* Bu verikümesi içerisinde olan kadın hasta kayıtlarının hamilelik sayısını görelim.Buna göre diabet teşisi hakkında bilgi edinebiliriz

In [ ]:
df["Pregnancies"].min()

In [ ]:
df["Pregnancies"].max()

* Hamilelik sayısı maximum 17 iken,hiç hamile olmayan kadınların da verileri tutulmuştur.Bunların sıklıklarını kontrol geldi.

In [ ]:
df["Pregnancies"].mean()

In [ ]:
df["Pregnancies"].value_counts()

* Bu kontrole göre en çok tek çocuk sahibi olan insanların kayıtları bu öznitelik içerisinde yer alıyor.

## 2.2 Glucose

* Glikoz seviyelerini kontrol edelim

In [ ]:
df[["Glucose"]]

* Glikoz seviyelerine göz gezdirdeikten sonra.Glikoz seviyeleri hakkında minimum ve maximum seviyeyi bulup,sıklıklarını ve ortalamalarını kontrol edelim

In [ ]:
df[["Glucose"]].min()

* Glikoz seviyesi 0 olan hastalar var.Bunlar birer anomali olabilir.Bu gibi durumları önişleme tarfında inceleyelim

In [ ]:
df[["Glucose"]].max()

In [ ]:
df["Glucose"].value_counts()

In [ ]:
df["Glucose"].nunique()

In [ ]:
df["Glucose"].mean()

* Glucose alanında bulunan anomali değerleri hedefe bağlı özelliklerle moda göre,veyahut doğrudan kolonun sahip olduğu ortalama ile doldurabiliriz

## 2.3. BloodPressure

In [ ]:
df["BloodPressure"].head()

* BloodPressure minimum 0, maksimum 122 olarak seyrediyor.Bunları düzenleyelim.

In [ ]:
df["BloodPressure"].min()

In [ ]:
df["BloodPressure"].max()

* Kan basıncı ile alakalı özniteliği inceleyelim.Burada blood pressure değeri 0 olarak görüntülenen kayıtlar mevut.Bu kayıtlar birer anomali olabilir.Bunları ön işleme tarafında çözmeye çalışalım

In [ ]:
df["BloodPressure"].value_counts()

* Kan basıncı en fazla 70 olan 57 hasta varken çok az olan değerlerde 40 veya çok yüksek değerlerde 122 gibi sıklık giderek azalıyor.Ortalama değerlerde sıklık fazla görülüyor olabilir.Bu özelliğin ortalama değerlerini inceleyelim

In [ ]:
df["BloodPressure"].mean()

In [ ]:
df["BloodPressure"].nunique()

## 2.4. SkinThickness

* Deri kalınlığı hakkında bilgi edinelim.Bu kolonun diabetle olan ilgisini araştıralım

In [ ]:
df["SkinThickness"].head()

* Burada da skinthickness ile ilgili minimum ve maksimum rakamlar hakkında bilgi edinelim.Sıklıklarını inceleyelim

In [ ]:
df[["SkinThickness"]].min()

In [ ]:
df[["SkinThickness"]].max()

In [ ]:
df[["SkinThickness"]].mean()

* Deri kalınlığı 0 olan 227 kayıt bulunuyor ve bu kayıtlar çok aykırı olabilir.Bu anlamsız bir değeri ifade ediyor olabilir.Bu değerleri nasıl güncelleyeceğimizi önişleme tarfında halledelim

In [ ]:
df["SkinThickness"].value_counts()

## 2.5. Insulin

In [ ]:
df[["Insulin"]].head()

In [ ]:
df[["Insulin"]].min()

In [ ]:
df[["Insulin"]].max()

In [ ]:
df[["Insulin"]].mean()

In [ ]:
df["Insulin"].value_counts()

In [ ]:
df["Insulin"].nunique()

* Insulin direnci 0 olarak görünen 374 kayıt var.Bu kayıtların nerelere vurduğunu inceleyelim

## 2.6. BMI

* BMI - vücut kitle indeksi ile ilgili olan kolon içerisinde BMI'da neler var,neler yok.Max-min değer aralığı nedir.Sıklıklarını kontrol edelim.Ortalamasını inceleyelim

In [ ]:
df[["BMI"]].head()

In [ ]:
df[["BMI"]].min()

In [ ]:
df["BMI"].max()

In [ ]:
df["BMI"].mean()

In [ ]:
df["BMI"].value_counts()

In [ ]:
df["BMI"].nunique()

## 2.7. DiabetesPedigreeFunction

In [ ]:
df[["DiabetesPedigreeFunction"]]

In [ ]:
df[["DiabetesPedigreeFunction"]].describe()

In [ ]:
df[["DiabetesPedigreeFunction"]].max()

In [ ]:
df[["DiabetesPedigreeFunction"]].mean()

In [ ]:
df["DiabetesPedigreeFunction"].nunique()

* Diabet pedigree fonksiyonu içerisinde 0.078 maksimum 2.42 olarak görünüyor.Ortalama değerini incelediğimizde 0.47 gibi ortalama fonksiyonel değerler buluyoruz

## 2.8. Age

In [ ]:
df[["Age"]].head()

In [ ]:
df[["Age"]].min()

In [ ]:
df[["Age"]].max()

In [ ]:
df[["Age"]].mean()

In [ ]:
df["Age"].value_counts()

* df'in sahip olduğu Yaş kolonu içerisinde minimum 21 kısıtı ile maximum 81 yaşında olan kadınların kayıtları bulunmaktadır.Yaş ortalaması bu veri kümesi içerisinde 33 gibi seyir etmektedir.En fazla 22 yaşındaki kadınların ilgili verileri bu veriseti içerisinde bulunuyor.

## 2.9. Outcome

* Outcome bize hedef kolonu hakkında gerekli bilgileri veriyor.Kaç tane diabeti olan hasta veya olmayan bireyler var bunların miktarlarını inceleyip.Sıklıklarını kontrol edelim

In [ ]:
df["Outcome"].head()

In [ ]:
df["Outcome"] = df["Outcome"].map(str)

* 1 ---> Diabeti olan
* 0 ---> Diabeti olmayan olarak gruplanıyor

In [ ]:
df["Outcome"].value_counts()

* Diabeti olan 268 kayıt varken, olmayan 500 tane kayıt bulunmaktadır.

# 3. Özniteliklerin İlşkisel Grafikleri - (Önişleme Öncesi)

# 3.1 Veri Önişleme

* Hedef değişkenin diğer kolonlarla olan ilşkisini kontrol edelim.Buradaki hedef ve diğer değişkenlerin birbiri ile korelasyonunu inceleyelim

In [ ]:
p = sns.pairplot(df, hue='Outcome')
p

## 3.1.1 DF İçerisindeki 0'ları Ortalama İle Değiştirme

### 3.1.1.1. Glucose

* Glikoz kolonunda bulunan 0 değerlerini gözlemleyelim

In [ ]:
df[df["Glucose"] == 0]

In [ ]:
df[df["Outcome"] == 0]["Glucose"].mean()

* Hedef kolonuna bağlı olarak en fazla 0 olan verilerden yola çıkarak glucose alanındaki verileri diabeti olmayan hastaların glikoz seviyelerinin ortalaması ile dolduralım

In [ ]:
df[df["Glucose"] == 0] = df[df["Glucose"] == 0].replace(0, 109.98)

In [ ]:
df[df["Glucose"] == 0]

### 3.1.1.2. BloodPressure

* Kan basıncı ile alakalı kolonda bulunan 0 değerlerini hedef kolonda seyreden kanbasıncı özelliklerinin ortalamasına göre dolduralım.bundan önce BloodPressure 0 olan değerleri görüntüleyelim

In [ ]:
df[df["BloodPressure"] == 0]

* Burada 0 olarak görünen veriler çok fazla.O yüzden hedef kolonunda 0 ve 1 değerlerinin toplam sayısını inceleyelim

In [ ]:
df[df["BloodPressure"] == 0]["Outcome"].value_counts()

* En fazla 0 olarak görünen diabeti olmayan hasta olduğu için.Bunlardan ortalamasını lıp içeridekileri replace edelim

In [ ]:
df[df["Outcome"] == 0]["BloodPressure"].mean()

In [ ]:
df[df["BloodPressure"] == 0] = df[df["BloodPressure"] == 0].replace(0, 68)

In [ ]:
df[df["BloodPressure"] == 0]

### 3.1.1.3 SkinThickness

In [ ]:
df[df["SkinThickness"] == 0]

In [ ]:
df[df["SkinThickness"] == 0]["Outcome"].value_counts()

* En fazla 0 olan veriler hedefte bulunduğu için hedef kolonun skinthickness ile olan bağlamından yola çıkarak ortalama değerlerini bulalım

In [ ]:
df[df["Outcome"] == 0]["SkinThickness"].mean()

In [ ]:
df[df["SkinThickness"] == 0] = df[df["SkinThickness"] == 0].replace(0, 20)

In [ ]:
df[df["SkinThickness"] == 0]

### 3.1.1.4 Insulin

In [ ]:
df[df["Insulin"] == 0]

In [ ]:
df[df["Insulin"] == 0]["Outcome"].value_counts()

* Insulin direncinde Hedef olarak diabeti olup olmayanalrı incelediğimizde olmayanların olanlardan daha fazla olduğu görülüyor.Bu durumda buradaki 0 değerlerinden yola çıkarak Insulin'in ortalamasını alalım

In [ ]:
df[df["Outcome"] == 0]["Insulin"].mean()

In [ ]:
df[df["Insulin"] == 0] = df[df["Insulin"] == 0].replace(0, 96)

In [ ]:
df[df["Insulin"] == 0]

### 3.1.1.5. BMI

In [ ]:
df[df["BMI"] == 0]

In [ ]:
df[df["Outcome"] == 0]["BMI"].mean()

In [ ]:
df[df["BMI"] == 0] = df[df["BMI"] == 0].replace(0, 31)

In [ ]:
df[df["BMI"] == 0]

* 0 olan alanlardaki verilerimizi doldurduktan sonra df'imizi görüntüleyelim

In [ ]:
df.head()

* Verilerimiz arasında önişleme tamamlandıktan sonraki korelasyonu görmek amacıyla yeni bir ısı haritası çıkaralım.
* Korelasyonların arasındaki ilşkiye bakalım

In [ ]:
f, ax = plt.subplots(figsize=[20, 15])
sns.heatmap(df.corr(), annot=True, ax=ax, cmap="Blues")

* df içerisinde bulunan korelasyona bağlı olarak,
    * 1.Glucose & BloodPressure arasında 22% pozitif korelasyon,
    * 2.Insulin & Glucose arasında 34% pozitif korelasyon,
    * 3.BMI & BloodPressure arasında 22% pozitif korelasyon,
    * 4.BMI & SkinThickness arasında 46% pozitif korelasyon,
    * 5.Age & BloodPressure 33% pozitif korelasyon,
    * 6.Age & Glucose arasında 27% pozitif korelasyon,
    * 7.Outcome & Pregnancies arasında 22% pozitif korelasyon bulunmaktadır

# 4. Outlier Değerler

In [ ]:
df.head()

## 4.1. Outlier Değerler Hakkında Bilgi

* Kolonların içerisinde outlier değerler hakkında genel bir bilgi edinelim

In [ ]:
fig, [[ax1, ax2, ax3], [ax4 ,ax5, ax6],
      [ax7, ax8, ax9]] = plt.subplots(nrows=3,
                                      ncols=3, figsize=(14, 20))
ax1.boxplot(df["Pregnancies"])
ax1.set_title("Pregnancies Outlier")
ax2.boxplot(df["Glucose"])
ax2.set_title("Glucose Outlier")
ax3.boxplot(df["BloodPressure"])
ax3.set_title("BloodPressure Outlier")
ax4.boxplot(df["SkinThickness"])
ax4.set_title("SkinThickness Outlier")
ax5.boxplot(df["Insulin"])
ax5.set_title("Insulin Outlier")
ax6.boxplot(df["BMI"])
ax6.set_title("BMI Outlier")
ax7.boxplot(df["DiabetesPedigreeFunction"])
ax7.set_title("DiabetesPedigreeFunction Outlier")
ax8.boxplot(df["Age"])
ax8.set_title("Age Outlier")

* Outlier değerler hakkında her bir kolon için ayrı ayrı box-plot üretip,histograma dökelim.Bu şekilde aykırı değerleri görüntülemiş oluruz

In [ ]:
def detect_outlier(data):
    q1 = np.quantile(data, 0.25)
    q3 = np.quantile(data, 0.75)
    IQR = q3 - q1
    low = q1 - (2 * IQR)
    high = q3 + (2 * IQR)
    return high, low

## 4.1.1 Pregnancies

* Outlier değerlerin altında ve üstünde kalan değerleri inceleyelim

In [ ]:
highp, bottomp = detect_outlier(df["Pregnancies"])

- Pregnancies kolonu içinde en yüksek limiti 16 ve en düşük limiti -9 olarak kutumuzu ayarladık

In [ ]:
highp

In [ ]:
bottomp

* Limitlerin üstünde ve altında kalan değerleri görüntüleyelim.Aşağıda da limitler arasında yaşam süren değerleri görüntüleyelim.(768, 9) 768 satır ve 9 tane özelliğimize göre inceleyelim

In [ ]:
df[df["Pregnancies"] > bottomp]

In [ ]:
df[df["Pregnancies"] < highp]

* Limit kısıtlarından outlier olarak saptananları görüntüleyelim

In [ ]:
df[(df["Pregnancies"] > bottomp) & (df["Pregnancies"] < highp)]

In [ ]:
df  = df[(df['Pregnancies'] > bottomp) & (df['Pregnancies'] < highp)]
df['Pregnancies'].plot(kind='box')
plt.figure()
df['Pregnancies'].plot(kind='hist', bins=50)

## 4.1.2 Glucose

In [ ]:
highg, bottomg = detect_outlier(df["Glucose"])

* Glikoz seviyesine bağlı olarak tanımlanan yüksek ve düşük değerleri saptayalım

In [ ]:
highg

In [ ]:
bottomg

* Glikoz kolonunda en yüksek ve en düşük limitler sırasıyla 218.25 ile 19.5 olarak saptanmıştır.Dataframe içerisinde en düşük limitten fazla, en büyük limitten daha az verileri df içerisinde görüntüleyelim

In [ ]:
df[df["Glucose"] > bottomg]

In [ ]:
df[df["Glucose"] < highg]

In [ ]:
df[(df["Glucose"] > bottomg) & (df["Glucose"] < highg)]

In [ ]:
df  = df[(df['Glucose'] > bottomg) & (df['Glucose'] < highg)]
df['Glucose'].plot(kind='box')
plt.figure()
df['Glucose'].plot(kind='hist', bins=50)

## 4.1.3 BloodPressure

In [ ]:
highbp, bottombp = detect_outlier(df["BloodPressure"])

- Boxplot'un sahip olduğu limitleri girelim

In [ ]:
highbp

In [ ]:
bottombp

* En düşük ve en yüksek kan basıncı sınırlarımızı 112.0 ile 32.0 olarak güncelledik

In [ ]:
df[df["BloodPressure"] > bottombp]

In [ ]:
df[df["BloodPressure"] < highbp]

In [ ]:
df[(df["BloodPressure"] > bottombp) & (df["BloodPressure"] < highbp)]

In [ ]:
df = df[(df["BloodPressure"] > bottombp) & (df["BloodPressure"] < highbp)]
df["BloodPressure"].plot(kind="box")
plt.figure()
df["BloodPressure"].plot(kind="hist", bins=50)

## 4.1.4 SkinThickness

In [ ]:
highst, bottomst = detect_outlier(df["SkinThickness"])

* Verilerimiz arasında df["SkinThickness"] kolonundaki verilerde uygulanacak boxplot sınırlarını görüntüleyelim

In [ ]:
highst

In [ ]:
bottomst

* Boxplot'un içeriğinde en yüksek 56 ve en düşük -4 değerlerini görüyoruz.Bunların sınırladığı alanın dışındakiler birer aykırı gözlem değeri olarak kabul ediliyor

In [ ]:
df[df["SkinThickness"] > bottomst]

In [ ]:
df[df["SkinThickness"] < highst]

In [ ]:
df[(df["SkinThickness"] > bottomst) & (df["SkinThickness"] < highst)]

In [ ]:
df = df[(df["SkinThickness"] > bottomst) & (df["SkinThickness"] < highst)]
df["SkinThickness"].plot(kind="box")
plt.figure()
df["SkinThickness"].plot(kind="hist", bins=100)

## 4.1.5 Insulin

In [ ]:
highin, bottomin = detect_outlier(df["Insulin"])

* Boxplot'un snırladığı en düşük ve en yüksek marker'ları görüntüleyelim

In [ ]:
df[df["Insulin"] > bottomin]

In [ ]:
df[df["Insulin"] < highin]

In [ ]:
df[(df["Insulin"] > bottomin) & (df["Insulin"] < highin)]

In [ ]:
df  = df[(df['Insulin'] > bottomin) & (df['Insulin'] < highin)]
df['Insulin'].plot(kind='box')
plt.figure()
df['Insulin'].plot(kind='hist', bins=100)

## 4.1.6 BMI

In [ ]:
highbmi, bottombmi = detect_outlier(df["BMI"])

In [ ]:
highbmi

In [ ]:
bottombmi

In [ ]:
df[df["BMI"] > bottombmi]

In [ ]:
df[df["BMI"] < highbmi]

In [ ]:
df[(df["BMI"] > bottombmi) & (df["BMI"] < highbmi)]

In [ ]:
df  = df[(df['BMI'] > bottombmi) & (df['BMI'] < highbmi)]
df['BMI'].plot(kind='box')
plt.figure()
df['BMI'].plot(kind='hist', bins=100)

## 4.1.7 DiabetesPedigreeFunction

In [ ]:
highdpf, bottomdpf = detect_outlier(df["DiabetesPedigreeFunction"])

In [ ]:
highdpf

In [ ]:
bottomdpf

In [ ]:
df[df["DiabetesPedigreeFunction"] > bottomdpf]

In [ ]:
df[df["DiabetesPedigreeFunction"] < highdpf]

In [ ]:
df[(df["DiabetesPedigreeFunction"] > bottomdpf) &
   (df["DiabetesPedigreeFunction"] < highdpf)]

In [ ]:
df  = df[(df['DiabetesPedigreeFunction'] > bottomdpf) &
         (df['DiabetesPedigreeFunction'] < highdpf)]
df['DiabetesPedigreeFunction'].plot(kind='box')
plt.figure()
df['DiabetesPedigreeFunction'].plot(kind='hist', bins=100)

## 4.1.8 Age

In [ ]:
highage, bottomage = detect_outlier(df["Age"])

In [ ]:
highage

In [ ]:
bottomage

In [ ]:
df[df["Age"] > bottomage]

In [ ]:
df[df["Age"] < highage]

In [ ]:
df[(df["Age"] > bottomage) & (df["Age"] < highage)]

In [ ]:
df  = df[(df['Age'] > bottomage) &
         (df['Age'] < highage)]
df['Age'].plot(kind='box')
plt.figure()
df['Age'].plot(kind='hist', bins=100)

* Şimdi verilerimiz outlier oluşturan kalabalık bilgilerden arındırılmış formatta.Bütün özelliklere ait boxplotları çizdirelim.Buna göre artık görselleştirmelerimizi yapabiliriz

In [ ]:
fig, [[ax1, ax2, ax3], [ax4 ,ax5, ax6],
      [ax7, ax8, ax9]] = plt.subplots(nrows=3,
                                      ncols=3, figsize=(14, 20))
ax1.boxplot(df["Pregnancies"])
ax1.set_title("Pregnancies Outlier")
ax2.boxplot(df["Glucose"])
ax2.set_title("Glucose Outlier")
ax3.boxplot(df["BloodPressure"])
ax3.set_title("BloodPressure Outlier")
ax4.boxplot(df["SkinThickness"])
ax4.set_title("SkinThickness Outlier")
ax5.boxplot(df["Insulin"])
ax5.set_title("Insulin Outlier")
ax6.boxplot(df["BMI"])
ax6.set_title("BMI Outlier")
ax7.boxplot(df["DiabetesPedigreeFunction"])
ax7.set_title("DiabetesPedigreeFunction Outlier")
ax8.boxplot(df["Age"])
ax8.set_title("Age Outlier")

In [ ]:
sns.pairplot(df, hue="Outcome")

* df içerisindeki temizlenen veriler sonrası korelasyonu inceleyelim

In [ ]:
f, ax = plt.subplots(figsize=[20, 15])
sns.heatmap(df.corr(), annot=True, ax=ax, cmap="BuPu")

* Burada bulunan verilerde 54% oranında SkinThickness ve BMI arasında,
* SkinThickness ve Insulin arasında 38% oranında,
* Age ve BloodPressure 37% arasında korelasyon bulunmaktadır

In [ ]:
df.shape

* Outlier değerler df içerisinden atılmış bulunmaktadır çünkü gözlemlerimiz 768 satırdan 646 satıra indi

# 5. Veri Görselleştirme

In [ ]:
df.head()

## 5.1. PieChart ve Histogram

### Outcome

* Hedef değişkenimiz içerisinde en fazla hasta olmayan 66,41% oranında insanların olduğunu bunula beraber 33,59% oranında hasta olan insanların da bulunduğunu görebiliyoruz

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (15, 7))
f.suptitle("Diabetes?", fontsize = 18.)
_ = df.Outcome.value_counts().plot.bar(ax = ax[0], rot = 0, color = (sns.color_palette("pastel")[4], sns.color_palette("pastel")[2])).set(xticklabels = ["No", "Yes"])
_ = df.Outcome.value_counts().plot.pie(labels = ("No", "Yes"), autopct = "%.2f%%", label = "", fontsize = 13., ax = ax[1],\
colors = (sns.color_palette("pastel")[4], sns.color_palette("pastel")[2]), wedgeprops = {"linewidth": 1.5, "edgecolor": "#F7F7F7"}), ax[1].texts[1].set_color("#F7F7F7"), ax[1].texts[3].set_color("#F7F7F7")

## 5.2. Violin Plots

In [ ]:
df.head()

### Bütün Öznitelikler

* Renk paleti için gerekli olan renk kodlarını içeren bir color tuple oluşturalım.Buna bağlı olarak tam 10 tane renk içeren Set3 paletinden aldığımız renk tuple kodlarını indexleyerek color değişkenine violinplot methodu içerisindeki değişkene değer olarak atayalım.Bu şekildeher bir değişkenin hedef ile olan ilşkisini ve hangi değerler arasında yoğunluk olduğunu tanımlayabiliriz

In [ ]:
qualitative_colors = sns.color_palette("Set3", 10)
sns.palplot(qualitative_colors)
type(qualitative_colors[0])

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(18, 18))
plt.suptitle("Hedef ile Özelliklerin Violin Grafikleri",
             fontsize=25)
sns.violinplot(x="Pregnancies", y="Outcome", data=df, ax=ax[0, 0],
               color=qualitative_colors[0])
sns.violinplot(x="Glucose", y="Outcome", data=df, ax=ax[0, 1],
               color=qualitative_colors[1])
sns.violinplot(x="BloodPressure", y="Outcome", data=df, ax=ax[1, 0],
               color=qualitative_colors[2])
sns.violinplot(x="SkinThickness", y="Outcome", data=df, ax=ax[1, 1],
               color=qualitative_colors[3])
sns.violinplot(x="Insulin", y="Outcome", data=df, ax=ax[2, 0],
               color=qualitative_colors[4])
sns.violinplot(x="BMI", y="Outcome", data=df, ax=ax[2, 1],
               color=qualitative_colors[5])
sns.violinplot(x="DiabetesPedigreeFunction", y="Outcome",
               data=df, ax=ax[3, 0], color=qualitative_colors[6])
sns.violinplot(x="Age", y="Outcome", data=df, ax=ax[3, 1],
               color=qualitative_colors[7])
plt.show()

## 5.3. Scatterplot

### Age - Insulin

In [ ]:
sns.scatterplot(x="Age", y="Insulin", data=df, hue="Outcome")
plt.show()

### SkinThickness - BMI

In [ ]:
sns.scatterplot(x="SkinThickness", y="BMI", data=df, hue="Outcome")
plt.show()

### Insulin - SkinThickness

In [ ]:
sns.scatterplot(x="SkinThickness", y="Insulin", data=df, hue="Outcome")
plt.show()

### BloodPressure - Age

In [ ]:
sns.scatterplot(x="BloodPressure", y="Age", data=df, hue="Outcome")
plt.show()

### Glucose - Insulin

In [ ]:
sns.scatterplot(x="Glucose", y="Insulin", data=df, hue="Outcome")
plt.show()

### Pregnancies - Age

In [ ]:
sns.scatterplot(x="Age", y="Pregnancies", data=df, hue="Outcome")
plt.show()

# 6. Sınıflandırma

* Bu veri kümesine bağlı olarak veri önişleme, outlier detection, görselleştirme ve modelleme kısımlarını oluşturduk.Bu kısımları artık geçtiğimize göre sınıflandırma üzerine çalışmaya başlayalım

## Veri Standardizasyon

* Veriyi standard hale getirmek.Aslında dataframe içerisinde df'i karartan yüksek büyüklükte,birimde değerlerin normalize etme işlemini uygulamak.Bu şekilde ham değerler elde ederek eğitim ve test için kullanılabilir formatta sağlıklı stanrdize edilmiş veriler tutabiliriz

In [ ]:
Image(filename="../input/images/Standardization.png")

* Standardizasyon, değerlerin ortalama etrafında bir birim standart sapma ile ortalandığı başka bir ölçeklendirme tekniğidir. Bu, özniteliğin ortalamasının sıfır olduğu ve ortaya çıkan dağılımın bir birim standart sapmaya sahip olduğu anlamına gelir.

* Nu, unsur değerlerinin ve Unsur ölçeklemesinin ortalamasıdır: Sigma, unsur değerlerinin standart sapmasıdır. Bu durumda, değerlerin belirli bir aralıkla sınırlı olmadığını unutmayın.

* Şimdi, aklınızdaki en büyük soru, normalleştirmeyi ne zaman kullanmalıyız ve standardizasyonu ne zaman kullanmalıyız?
Verilerinizin dağılımının bir Gauss dağılımını takip etmediğini bildiğinizde normalleştirme kullanmak iyidir. Bu, K-En Yakın Komşular ve Sinir Ağları gibi verilerin herhangi bir dağıtımını varsaymayan algoritmalarda yararlı olabilir.
Öte yandan standardizasyon, verilerin bir Gauss dağılımını takip ettiği durumlarda yardımcı olabilir. Ancak bunun mutlaka doğru olması gerekmez. Ayrıca, normalleştirmenin aksine, standardizasyonun bir sınır aralığı yoktur. Dolayısıyla, verilerinizde aykırı değerler olsa bile, bunlar standardizasyondan etkilenmeyecektir.

In [ ]:
df_copy = df.copy()

In [ ]:
sc = StandardScaler()

In [ ]:
X =  pd.DataFrame(sc.fit_transform(df_copy.drop(["Outcome"],
                                                axis = 1),),
        columns=['Pregnancies', 'Glucose',
                 'BloodPressure', 'SkinThickness',
                 'Insulin', 'BMI',
                 'DiabetesPedigreeFunction', 'Age'])

In [ ]:
X.head()

* Hedef degiskeni bir y değerine atayalım.Train ve test aşaması için gerekli olacaktır

In [ ]:
y = df_copy["Outcome"]
y.head()

# 7. Eğitim-Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1/3, random_state=42, stratify=y)

* test ve eğitim için gerekli ik tane boş liste oluşturalım

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

## Naive Bayes

* Naive Bayes bir eleman için her durumun olasılığını hesaplar.Olasılık değeri en yüksek olana göre sınıflandırma yapar.Tahmin edemediği bölgelere 0 atar.Bunu da laplace yöntemi ile çözebiliriz

In [ ]:
Image(filename="../input/images/Bayes.png")

* BernoulliNB : Tahmin edeceğiniz veri veya kolon ikili ise bunu kullanabiliriz


In [ ]:
bnb = BernoulliNB()

**Test ve Traine göre Tahmin**

In [ ]:
bnb.fit(X_train, y_train.ravel())
result = bnb.predict(X_test)

**Karmaşıklık matrisi**

1. [0, 0] -- FN
2. [0, 1] -- TN
3. [1, 0] -- TP
4. [1, 1] -- TN

In [ ]:
cm = confusion_matrix(y_test, result)
cm

In [ ]:
accuracy = accuracy_score(y_test, result)
accuracy

* Naive Bayes algoritmasının doğruluk oranı 76,38% olarak çıkmıştır.Çok başarılı bir sonuç değildir.

## SVM

SVM aslında iki farklı veri grubuna bağlı olarak iki grubu bir en iyi çizgi doğrusu ile ayırmaya çalışır.SVM aynı zamanda doğrusal ve doğrusal olmayan verileride sınıflandırabilir ancak genellikle verileri doğrusal olarak sınıflandırmaya çalışır.SVM ile amaçladığımız şey ise tamamen bir düzelm ile ayırdığımız veri gruplarını sınıflandırabilmektir.Buna Kernel Trick yöntemi denir

In [ ]:
Image(filename="../input/images/SVM.png")

In [ ]:
svc = SVC(kernel="linear")

**Eğitim**

In [ ]:
svc.fit(X_train, y_train)
result = svc.predict(X_test)

**Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_test, result)
cm

**Başarı Oranı**

In [ ]:
accuracy = accuracy_score(y_test, result)
accuracy

kernel içerisine verdiğim "‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’" parametrelerinden en fazla doğruluk değeri oluşturan linear değeri olduğu için onu notebookuma alamak istedim

## KNN ( K nearest neigborhood)

K-NN non-parametric ( parametrik olmayan ), lazy ( tembel ) bir öğrenme algoritmasıdır. lazy kavramını anlamaya çalışırsak eager learning aksine lazy learning’in bir eğitim aşaması yoktur. Eğitim verilerini öğrenmez, bunun yerine eğitim veri kümesini “ezberler”. Bir tahmin yapmak istediğimizde, tüm veri setinde en yakın komşuları arar.Algoritmanın çalışmasında bir K değeri belirlenir. Bu K değerinin anlamı bakılacak eleman sayısıdır. Bir değer geldiğinde en yakın K kadar eleman alınarak gelen değer arasındaki uzaklık hesaplanır. Uzaklık hesaplama işleminde genelde Öklid fonksiyonu kullanılır. Öklid fonksiyonuna alternatif olarak Manhattan, Minkowski ve Hamming fonksiyonlarıda kullanılabilir. Uzaklık hesaplandıktan sonra sıralanır ve gelen değer uygun olan sınıfa atanır.

In [ ]:
Image(filename="../input/images/KNN.png")

**Eğitim**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, metric="minkowski")
knn.fit(X_train, y_train.ravel())
result = knn.predict(X_test)

**Karmaşıklık Matrisi**

In [ ]:
cm = confusion_matrix(y_test, result)
cm

**Başarı Oranı**

In [ ]:
accuracy = accuracy_score(y_test, result)
accuracy

## Decision Tree

Decision Tree algoritması çok sınıflı verilerin kullanımında ve sonuç çıkarımlarında kullanılır. Sayısal veya sayısal olmayan değerler üzerinde işlemler yapar.

Mantığına değinecek olursam sınıfların entropi değerlerine göre ağaç üzerinde dizilimi ve daha sonra kontrol edilmek istenen değerin root node’dan verilip dallarda ilerlemesi ile sonuca ulaşılması şekline çalışır. Train veri setimize göre ağacın yapısı değişir ve şekillenir.Ağacın root düğümü nasıl seçilir diye soracak olursanız tüm sınıfların entropi değerleri hesaplanır ve genel entropy’den çıkarılır çıkan entropyler arasında en büyük olan root node kabul edilir ve kendi içerisinde dallanır. Daha sonra dallanan alt nesnelerin entropisi hesaplanır eğer 0’dan farklı bir değer çıkmış ise o daldan yeniden bir alt dal oluşturulmak zorundadır. Tekrar sınıfların entropisi hesaplanarak en büyük olan alt dala eklenir ve işlem böyle devam eder. En alt dallarda işlem ZeroR algoritması gibi en çok olan adedi bulunan değer sonuç kabul edilir.

In [ ]:
Image(filename="../input/images/DT.png")

**Eğitim**

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
clf = clf.fit(X_train, y_train)
result = clf.predict(X_test)

**Karmaşıklık Matrisi**

In [ ]:
cm = confusion_matrix(y_test, result)
cm

In [ ]:
accuracy = accuracy_score(y_test, result)
accuracy

# 8. Sonuç

##### Çıkan sonuçlar her ne kadar tatminkar olmasa da bu veri kümesi için en iyi çalışan algoritmalardan biri SVM'dir diyebiliriz
1. SVM --> 76,85%
2. Naive Bayes --> 76,38%
3. KNN --> 74,07%
4. DecisionTree --> 66,66%